In [ ]:
import os
os.getcwd()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [11]:

#os.chdir('/content/drive/MyDrive/model')

In [ ]:

os.getcwd()

In [13]:

import sys
if not 'CLM_Former' in sys.path:
    sys.path += ['CLM_Former']

# Experiments: Train and Test

In [14]:
from utils.tools import dotdict
from exp.exp_main import Exp_Main
import torch
import os

In [ ]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main#exp stands for experiments
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')

# model define
parser.add_argument('--bucket_size', type=int, default=4, help='for Reformer')
parser.add_argument('--n_hashes', type=int, default=4, help='for Reformer')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size')
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in encoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')

# args = parser.parse_args()
args = dotdict()
args.target = 'OT'
args.des = 'test'
args.dropout = 0.05
args.num_workers = 10
args.gpu = 0
args.lradj = 'type1'
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False
args.freq = 'h'
args.checkpoints = './checkpoints/'
args.bucket_size = 4
args.n_hashes = 4
args.is_trainging = True
args.root_path = './data/electricity/'
args.data_path ='electricity.csv'
args.model_id='New_electricity_128_192' 
args.model = 'CLM_Former'
args.data = 'custom'
args.features = 'M'    # M, MS, S
args.seq_len = 128
args.label_len = 48
args.pred_len = 192   # 96, 192, 336
args.e_layers = 4
args.d_layers = 4
args.n_heads = 8
args.factor = 3
args.enc_in = 321
args.dec_in =321
args.c_out = 321
args.d_model = 512
args.des = 'Exp'
args.itr = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 1
args.distil = True
args.output_attention = False
args.patience= 3
args.learning_rate = 0.0001
args.batch_size = 64
args.embed = 'timeF'
args.activation = 'gelu'
args.use_amp = False
args.loss = 'mse'
args.train_epochs = 20
print('Args in experiment:')
print(args)

Exp = Exp_Main

In [ ]:
import time
import torch
import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

train_times = []
peak_memory_mb = []

for ii in range(args.itr):
    
    setting = 'New_CLM_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.model_id, args.model, args.data, args.features,
        args.seq_len, args.label_len, args.pred_len,
        args.d_model, args.n_heads, args.e_layers, args.d_layers,
        args.d_ff, args.factor, args.embed, args.distil, args.des, ii)

    exp = Exp(args)  

    print(f'\n>>>>>>> Start training : {setting} >>>>>>>>>>>>>>>>>>>>>>>>>>\n')

    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats(device=args.gpu)

    start_train_time = time.time()
    exp.train(setting)
    end_train_time = time.time()

    current_train_time = end_train_time - start_train_time
    train_times.append(current_train_time)

    current_peak_mem = torch.cuda.max_memory_allocated(device=args.gpu) / (1024 ** 2)
    peak_memory_mb.append(current_peak_mem)

    print(f"Training Time: {current_train_time:.2f} seconds")
    print(f"Peak GPU Memory: {current_peak_mem:.2f} MB")

    torch.cuda.empty_cache()

print(f"\n--- Training Results across {args.itr} runs ---")
print(f"Training Time (Mean ± Std): {np.mean(train_times):.2f} ± {np.std(train_times):.2f} s")
print(f"Peak Memory (Mean ± Std):   {np.mean(peak_memory_mb):.2f} ± {np.std(peak_memory_mb):.2f} MB")


In [ ]:
import torch
import time
import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


inference_times = []
peak_memory_mb = []


for ii in range(args.itr):
    setting = 'New_CLM_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.model_id, args.model, args.data, args.features,
        args.seq_len, args.label_len, args.pred_len,
        args.d_model, args.n_heads, args.e_layers, args.d_layers,
        args.d_ff, args.factor, args.embed, args.distil, args.des, ii)

    print(f"\n>>>>>>> Start testing : {setting} <<<<<<<<<<<<<<<<<<<<<<<<<<<\n")

    exp = Exp(args)

    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats(device=device)

    model_path = f'./checkpoints/{setting}/best_model.pth'
    assert os.path.exists(model_path), f"❌ Model file not found: {model_path}"

    print(f"✅ Loading best model from {model_path}")
    checkpoint = torch.load(model_path, map_location=device)
    exp.model.load_state_dict(checkpoint)
    exp.model.to(device)
    exp.model.eval()

    start_test_time = time.time()
    avg_mae, avg_mse, avg_rmse, avg_mape, avg_mspe, total_time, peak_mem = exp.test(setting, test=1)
    end_test_time = time.time()

    current_test_time = end_test_time - start_test_time
    inference_times.append(current_test_time)

    current_peak_mem = torch.cuda.max_memory_allocated(device=device) / (1024 ** 2)
    peak_memory_mb.append(current_peak_mem)

    print(f"Inference Time: {current_test_time:.2f} seconds")
    print(f"Peak GPU Memory: {current_peak_mem:.2f} MB")

    torch.cuda.empty_cache()

print(f"\n--- Final Test Results across {args.itr} runs ---")
print(f"Inference Time (Mean ± Std):  {np.mean(inference_times):.2f} ± {np.std(inference_times):.2f} s")
print(f"Peak Memory (Mean ± Std):     {np.mean(peak_memory_mb):.2f} ± {np.std(peak_memory_mb):.2f} MB")
